# 02 - Feature Engineering & Sampling

This notebook expands the dataset, defines the target variable `prepay`, and performs initial data exploration. Applied simple sampling and create baseline feature summaries for modeling. 

## Objectives:
- Load loan-level CRT data from multiple years
- Create 20% sample of loans from a baseline reporting period
- Define the binary `prepay` flag
- Conduct exploratory analysis on categorical features
- Identify candidate variables for modeling

## Summary
This notebook prepares a modeling-ready dataset by joining and sampling historical loan records, labeling the target variable, and performing initial feature analysis. We now have a clean dataset with labeled outcomes, ready for macroeconomic feature integration.

1. get more data in
2. [optional] simple sampling
3. start to explore relationship between prepay and other predictor variables: correlation, bin plot Current Actual UPB: divide UPB by 0 to100K, 100k to 200k, 200k to 300k, 300k+, plot the binned UPB against prepay.
4. produce prepay percentage in groupby statement
5. get interest rate data: get 10 year treasury rate from 2017 to 2023

In [2]:
import pandas as pd
import numpy as np

In [1]:
from os import listdir
from os.path import isfile, join

In [ ]:
file_loc='/Users/Downloads/CAS-102013-082023'

In [5]:
headerfile='CRT_Header_File.csv'
csv_files = [f for f in listdir(file_loc) if isfile(join(file_loc,f)) and (
    '2018.csv' in f or '2019.csv' in f or '2020.csv' in f)] 

In [6]:
header=pd.read_csv(file_loc+'/CRT_Header_File.csv')
df_list = []
for i,csv in enumerate(csv_files):
    df1=pd.read_csv(file_loc+'/'+csv, delimiter='|', header=None)
    df1.columns=header.columns
    df_list.append(df1)

/var/folders/nb/cq9dyksn0z39zq6k8k66f0xc0000gn/T/ipykernel_55484/3016847529.py:4: DtypeWarning: Columns (42,101) have mixed types. Specify dtype option on import or set low_memory=False.
  df1=pd.read_csv(file_loc+'/'+csv, delimiter='|', header=None)
/var/folders/nb/cq9dyksn0z39zq6k8k66f0xc0000gn/T/ipykernel_55484/3016847529.py:4: DtypeWarning: Columns (42,82,101) have mixed types. Specify dtype option on import or set low_memory=False.
  df1=pd.read_csv(file_loc+'/'+csv, delimiter='|', header=None)
/var/folders/nb/cq9dyksn0z39zq6k8k66f0xc0000gn/T/ipykernel_55484/3016847529.py:4: DtypeWarning: Columns (42,101) have mixed types. Specify dtype option on import or set low_memory=False.
  df1=pd.read_csv(file_loc+'/'+csv, delimiter='|', header=None)
/var/folders/nb/cq9dyksn0z39zq6k8k66f0xc0000gn/T/ipykernel_55484/3016847529.py:4: DtypeWarning: Columns (42,82) have mixed types. Specify dtype option on import or set low_memory=False.
  df1=pd.read_csv(file_loc+'/'+csv, delimiter='|', header=

### Load & Combine Multiple Years of CRT Data
Loaded all 2018–2020 datasets and apply the correct column headers.

In [7]:
df=pd.concat(df_list)

In [8]:
df.shape

(6714900, 108)

In [22]:
df.describe()

,Reference Pool ID,Loan Identifier,Monthly Reporting Period,Original Interest Rate,Current Interest Rate,Original UPB,UPB at Issuance,Current Actual UPB,Original Loan Term,Origination Date,...,Next Payment Change Date,Index,ARM Cap Structure,Initial Interest Rate Cap Up Percent,Periodic Interest Rate Cap Up Percent,Lifetime Interest Rate Cap Up Percent,Mortgage Margin,ARM Plan Number,Alternative Delinquency Resolution Count,Total Deferral Amount
count,6714900.0,6.714900e+06,6.714900e+06,6.714900e+06,5.729326e+06,6.714900e+06,6.714900e+06,6.714900e+06,6.714900e+06,6.714900e+06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4835.0,4835.000000
mean,1501.0,9.011017e+07,6.701883e+04,4.377809e+00,4.366868e+00,2.438257e+05,2.409014e+05,1.963082e+05,3.594334e+02,5.677707e+04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,5669.177431
std,0.0,6.294863e+04,3.452029e+04,3.660536e-01,3.639466e-01,1.229430e+05,1.222127e+05,1.361970e+05,5.770644e+00,1.313858e+04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3848.635625
min,1501.0,9.000000e+07,1.201800e+04,3.125000e+00,3.125000e+00,1.200000e+04,5.390770e+03,0.000000e+00,2.410000e+02,1.201700e+04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,387.720000
25%,1501.0,9.005547e+07,3.951850e+04,4.125000e+00,4.125000e+00,1.480000e+05,1.460381e+05,1.022023e+05,3.600000e+02,5.201700e+04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3083.680000
50%,1501.0,9.011095e+07,6.701900e+04,4.330000e+00,4.250000e+00,2.230000e+05,2.193503e+05,1.845485e+05,3.600000e+02,6.201700e+04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,4713.240000
75%,1501.0,9.016637e+07,9.451950e+04,4.625000e+00,4.625000e+00,3.210000e+05,3.178772e+05,2.840847e+05,3.600000e+02,6.201700e+04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,7185.400000
max,1501.0,9.021463e+07,1.220190e+05,6.125000e+00,6.125000e+00,1.223000e+06,1.218732e+06,1.218732e+06,3.600000e+02,1.220160e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,41300.990000


## create a 20% sample

In [10]:
df['reporting_period'] = pd.to_datetime(df['Monthly Reporting Period'].astype(str), format='%m%Y')

In [11]:
sample_loans = df[df['reporting_period']==pd.Timestamp('2017-11-1')].sample(frac=0.2)

In [12]:
sample_loans.shape

(37305, 109)

In [13]:
dfs=df.merge(sample_loans['Loan Identifier'], on='Loan Identifier', how='inner')

In [14]:
dfs.shape

(1342980, 109)

In [24]:
df.groupby('reporting_period').size()

reporting_period
2017-11-01    186525
2017-12-01    186525
2018-01-01    186525
2018-02-01    186525
2018-03-01    186525
2018-04-01    186525
2018-05-01    186525
2018-06-01    186525
2018-07-01    186525
2018-08-01    186525
2018-09-01    186525
2018-10-01    186525
2018-11-01    186525
2018-12-01    186525
2019-01-01    186525
2019-02-01    186525
2019-03-01    186525
2019-04-01    186525
2019-05-01    186525
2019-06-01    186525
2019-07-01    186525
2019-08-01    186525
2019-09-01    186525
2019-10-01    186525
2019-11-01    186525
2019-12-01    186525
2020-01-01    186525
2020-02-01    186525
2020-03-01    186525
2020-04-01    186525
2020-05-01    186525
2020-06-01    186525
2020-07-01    186525
2020-08-01    186525
2020-09-01    186525
2020-10-01    186525
dtype: int64

## Data exploration

In [27]:
dfs.describe()
dfs.describe().to_csv('vars_numeric.csv')

In [28]:
dfs.describe(include=[object])
dfs.describe(include=[object]).to_csv('vars_obj.csv')

### Define Target Variable: `prepay`
A loan is labeled as `prepay = 1` if it was prepaid (Zero Balance Code == 1), and `0` otherwise.

In [19]:
dfs['prepay']=0
dfs.loc[dfs['Zero Balance Code'] == 1, 'prepay'] = 1

In [20]:
new_cols=[]
for col in dfs.columns:
    new_cols.append(col.strip())
dfs.columns=new_cols

### Explore Categorical Predictors
Group by loan-related variables (e.g., Loan Purpose) to understand distribution and `prepay` behavior.

In [21]:
groupvars=['Loan Purpose']
dfs.groupby(groupvars,dropna=False).agg({'prepay':['count','mean']})

prepay          
               count      mean
Loan Purpose                  
C             331694  0.187718
P             843084  0.124453
R             168202  0.165081

# Next steps

1. have a list of variables that can be used as predictor
2. derive rate incentive, and house price change
3. go ahead and fit a tree model (training, out-of-sample test, out-of-time test)


## variables to exclude:
loan ID,
monthly reporting period,
zero balance code / date

In [46]:
vars = pd.read_excel('variable-selection.xlsx', sheet_name='numeric', index_col=None)
vars_selected_num=vars[vars['Selected'].isnull()]
vars_selected_num
# vars['Selected']

,var-name,count,mean,std,min,0.25,0.5,0.75,max,Selected
3,Original Interest Rate,1342980,4.377466,0.366255,3.125,4.1250,4.275,4.625,6.125,NaN
4,Current Interest Rate,1146712,4.366241,0.364105,3.125,4.1250,4.250,4.625,6.125,NaN
5,Original UPB,1342980,243164.830452,122426.579022,12000.000,148000.0000,222000.000,320000.000,1223000.000,NaN
6,UPB at Issuance,1342980,240209.340270,121778.616839,8539.260,145819.4600,219127.600,317148.070,1218732.090,NaN
7,Current Actual UPB,1342980,196020.161730,135640.407004,0.000,102150.0075,184864.635,283451.140,1218732.090,NaN
8,Original Loan Term,1342980,359.403780,5.925368,264.000,360.0000,360.000,360.000,360.000,NaN
9,Origination Date,1342980,56710.204637,13112.483754,12017.000,52017.0000,52017.000,62017.000,122016.000,NaN
10,First Payment Date,1342980,76740.495510,13118.471480,32017.000,72017.0000,72017.000,82017.000,102017.000,NaN
11,Loan Age,1146562,20.608680,10.176305,2.000,12.0000,20.000,29.000,44.000,NaN
12,Remaining Months to Legal Maturity,1146562,338.929312,12.249852,224.000,331.0000,340.000,348.000,481.000,NaN


In [47]:
vars = pd.read_excel('variable-selection.xlsx', sheet_name='object', index_col=None)
vars_selected_obj=vars[vars['Selected'].isnull()]
vars_selected_obj

,var-name,count,unique,top,freq,Selected
0,Channel,1342980,3,R,765793.0,NaN
1,Seller Name,1342980,24,Other,613980.0,NaN
2,Servicer Name,1146562,33,Other,415691.0,NaN
4,First Time Home Buyer Indicator,1342980,2,N,1084428.0,NaN
5,Loan Purpose,1342980,3,P,843084.0,NaN
6,Property Type,1342980,5,SF,800640.0,NaN
7,Occupancy Status,1342980,3,P,1098792.0,NaN
8,Property State,1342980,53,CA,187704.0,NaN
13,Modification Flag,1146580,2,N,1145729.0,NaN
15,Servicing Activity Indicator,1146562,2,N,1126848.0,NaN


In [48]:
var_list = vars_selected_num['var-name'].tolist() + vars_selected_obj['var-name'].tolist()
var_list

['Original Interest Rate',
 'Current Interest Rate',
 'Original UPB',
 'UPB at Issuance',
 'Current Actual UPB',
 'Original Loan Term',
 'Origination Date',
 'First Payment Date',
 'Loan Age',
 'Remaining Months to Legal Maturity',
 'Remaining Months To Maturity',
 'Maturity Date',
 'Original Loan to Value Ratio (LTV)',
 'Original Combined Loan to Value Ratio (CLTV)',
 'Number of Borrowers',
 'Debt-To-Income (DTI)',
 'Borrower Credit Score at Origination',
 'Co-Borrower Credit Score at Origination',
 'Number of Units',
 'Current Loan Delinquency Status',
 'UPB at the Time of Removal',
 'Scheduled Principal Current',
 'Total Principal Current',
 'Unscheduled Principal Current',
 'Last Paid Installment Date',
 'Foreclosure Date',
 'Disposition Date',
 'Foreclosure Costs',
 'Property Preservation and Repair Costs',
 'Asset Recovery Costs',
 'Miscellaneous Holding Expenses and Credits',
 'Associated Taxes for Holding Property',
 'Net Sales Proceeds',
 'Credit Enhancement Proceeds',
 'Repur